In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport

### Part 1. DATASET PRESENTATION

<b>A. <u>Loading</u></b>

In [ ]:
data = pd.read_csv('framingham.csv')

In [ ]:
data.head()

In [ ]:
data.shape

The dataset includes 4238 rows and 16 columns.

In [ ]:
data.info()

<b>B) <u>Source</u></b>

<p>The dataset is publically available on the Kaggle website, and it is from an ongoing cardiovascular study on residents of the town of Framingham, Massachusetts.</p>
<p>The classification goal is to predict whether the patient has 10-year risk of future coronary heart disease (CHD).</p>
<p>The dataset provides the patients’ information. Each variable represents a potential risk factor of contracting a coronary heart disease. There are both socio-demographic and medical risk factors.</p>

<b>C) <u>Features</u></b>

- Renaming columns

In [ ]:
data.columns = ['sex', 'age', 'education', 'smoker', 'cigs_per_day', 'bp_meds', 'prev_stroke', 'prev_hyp', 'diabetes', 'level_chol', 'bp_sys', 'bp_dias', 'bmi', 'heart_rate', 'level_gluc', 'ten_year_risk_chd']

In [ ]:
data.head()

- Features presentation

In [ ]:
features_presentation = pd.DataFrame(
    
{"Variables": data.columns, 
 
"Description": 
["0 : Female ; 1 : Male",
"Age at exam time",
"1 : Some High School ; 2 : High School or GED ; 3 : Some College or Vocational School ; 4 : College",
"0 : Non-current smoker ; 1: Current smoker",
"Number of cigarettes smoked per day (estimated average)",
"0 : Not on Blood Pressure medications ; 1 : Is on Blood Pressure medications",
"0 : No prevalent stroke ; 1 : Prevalent stroke(s)",
"0 : No prevalent hypertension ; 1 : Prevalent hypertension",
"0 : Non-diabetic patient ; 1 : diabetic patient",
"Total cholesterol level (mg/dL)",
"Systolic blood pressure (mmHg)",
"Diastolic blood pressure (mmHg)", 
"Body Mass Index (calculated as: Weight(kg) / Height(meter-squared))", 
"Heart rate (beats per minute)",
"Glucose level (mg/dL)",
"10-year risk of coronary heart disease (0 : no ; 1 : yes)"],

"Risk Factor Category": ["Socio-Demographic", "Socio-Demographic", "Socio-Demographic", "Socio-Demographic", "Behavioral", "Medical (history)", "Medical (history)", "Medical (history)", "Medical (history)", "Medical (measures)", "Medical (measures)", "Medical (measures)", "Medical (measures)", "Medical (measures)", "Medical (measures)", "Risk Prediction"], 

"Variable Type": ["Nominal", "Continuous", "Nominal", "Nominal", "Continuous", "Nominal", "Nominal", "Nominal", "Nominal", "Continuous", "Continuous", "Continuous", "Continuous", "Continuous", "Continuous", "Target : Nominal"]
})

In [ ]:
pd.set_option('max_colwidth', None)
features_presentation

### Part 2. PREPROCESSING

<b>A) <u>Missing values</u></b>

In [ ]:
data.isnull().sum()

- Education column

In [ ]:
mode_education = data['education'].mode()
mode_education

In [ ]:
data['education'].fillna(mode_education[0], inplace=True)
data['education'].isnull().sum()

- Cigs_per_day column

In [ ]:
median_cigs_per_day = data[data['smoker']==1]['cigs_per_day'].median()
median_cigs_per_day

In [ ]:
data['cigs_per_day'].fillna(median_cigs_per_day, inplace=True)
data['cigs_per_day'].isnull().sum()

- BP medication column

In [ ]:
mode_bp_meds = data['bp_meds'].mode()
mode_bp_meds

In [ ]:
data['bp_meds'].fillna(mode_bp_meds[0], inplace=True)
data['bp_meds'].isnull().sum()

- Level cholesterol column

In [ ]:
mean_level_chol = data['level_chol'].mean()
mean_level_chol

In [ ]:
data['level_chol'].fillna(mean_level_chol, inplace=True)
data['level_chol'].isnull().sum()

- BMI column

In [ ]:
mean_bmi = data['bmi'].mean()
mean_bmi

In [ ]:
data['bmi'].fillna(mean_bmi, inplace=True)
data['bmi'].isnull().sum()

- Heart rate column

In [ ]:
mean_heart_rate = data['heart_rate'].mean()
mean_heart_rate

In [ ]:
data['heart_rate'].fillna(mean_heart_rate, inplace=True)
data['heart_rate'].isnull().sum()

- Level glucose column

In [ ]:
mean_level_gluc = data['level_gluc'].mean()
mean_level_gluc

In [ ]:
data['level_gluc'].fillna(mean_level_gluc, inplace=True)
data['level_gluc'].isnull().sum()

<b>B) <u>Feature engineering</u></b>

<p>Variables 'smoker' and 'cigs_per_day' are redundant.</p>
<p>We can create a multi-level categorical variable 'tobacco' where:</p>

- 0 for non_smokers 
- 1 for light smokers (cigs_per_day <= 20)
- 2 for heavy smokers (cigs_per_day > 20)

In [ ]:
def smoker(row):
    if row['cigs_per_day'] == 0:
        return 0
    elif row['cigs_per_day'] <= 20:
        return 1
    else:
        return 2

data['tobacco'] = data.apply(lambda row: smoker(row), axis=1)

In [ ]:
data['tobacco'].value_counts()

In [ ]:
data.drop(columns=['smoker','cigs_per_day'], axis=1, inplace=True) 

<b>C) <u>Reorder columns</u></b>

In [ ]:
socio_demographic_vars = ['sex', 'age', 'education', 'tobacco']
med_history_vars = ['bp_meds', 'prev_stroke', 'prev_hyp', 'diabetes']
med_measures_vars = ['bmi', 'heart_rate', 'level_gluc', 'level_chol', 'bp_sys', 'bp_dias']
prediction_var = ['ten_year_risk_chd']

In [ ]:
new_order = socio_demographic_vars + med_history_vars + med_measures_vars + prediction_var
data = data[new_order]
data.head()

In [ ]:
data.info()

### Part 2. Exploratory Data Analysis (EDA)

<b>A) <u>Summary statistics</u></b>

In [ ]:
data.describe()

<b>B) <u>Profiling report</u></b>

In [ ]:
import pandas_profiling
data.profile_report()

<b>C) <u>Distribution of Target Variable : 10-year risk CHD</u></b>

In [ ]:
target_count = data['ten_year_risk_chd'].value_counts()
print('Patients not a risk:', target_count[0])
print('Patients at risk:', target_count[1])
print('Proportion:', round(target_count[0] / target_count[1], 2), ': 1')

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=False, figsize=(15,8))

ax1 = target_count.plot(kind='pie', startangle=60, autopct='%1.1f%%', labels=["Not at risk","At risk"], colors=['forestgreen', 'firebrick'], ax=ax1)
ax1.set(title = 'Percentage of patients at risk CHD')


ax2 = sns.countplot(x=data['ten_year_risk_chd'], palette=['forestgreen', 'firebrick'], ax=ax2)
ax2.set(xlabel='Risk CHD No / Yes', ylabel='Count', title='Count of patients according to risk CHD')


plt.show()

<p>The dataset has clearly an imbalanced distribution.<br>
Indeed, observations in the class 0 (patients not at risk) are much higher than the observations in the class 1 (patients at risk).<br>
The ratio is 1 to 5.5. <br>
There are techniques for handling imbalanced datasets : the 'SMOTE algorithm' for example (see "Part 3. Model Preparation")</p>

<b>D) <u>Socio-Demographic Variables</u></b>

- Age

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
ax = sns.countplot(x=data['age'], hue=data['ten_year_risk_chd'], palette=['forestgreen', 'firebrick'])
ax.set(xlabel='Age', ylabel='Count', title='Age vs Risk CHD')
plt.show()

- Sex

In [ ]:
fig = plt.subplots(figsize=(8,12))
plt.subplot(211)
data['sex'].value_counts().plot(kind='pie', startangle=90, autopct='%1.1f%%', colors=['pink', 'lightblue'], labels=['Female', 'Male'])
plt.title('Sex distribution')
plt.subplot(212)
sns.countplot(x=data['sex'], hue=data['ten_year_risk_chd'], palette=['forestgreen', 'firebrick']) 
plt.xlabel('Sex : 0=Female ; 1=Male')
plt.ylabel('Count')
plt.title('Risk CHD by Sex')
plt.show()

- Education

In [ ]:
fig = plt.subplots(figsize=(8,12))
plt.subplot(211)
data['education'].value_counts().plot(kind='pie', startangle=90, autopct='%1.1f%%', colors=['lemonchiffon', 'khaki', 'gold', 'goldenrod'], labels=['1: Some High School', '2: GED', '3: Some College', '4: College Graduate'])
plt.ylabel('')
plt.title('Distribution of Education levels')
plt.subplot(212)
sns.countplot(x=data['education'], hue=data['ten_year_risk_chd'], palette=['forestgreen', 'firebrick']) 
plt.xlabel('Education level')
plt.ylabel('Count')
plt.title('Risk CHD by Education level')
plt.show()

We can see that there is no correlation between the level of education and the risk to contract a CHD.<br>
The coefficient of correlation is very low : -0.05<br>
Therefore we can drop the 'education' column of the dataset.

In [ ]:
data = data.drop(['education'], axis=1)

- Tobacco 

In [ ]:
fig = plt.subplots(figsize=(8,12))
plt.subplot(211)
data['tobacco'].value_counts().plot(kind='pie', startangle=90, autopct='%1.1f%%', colors=["bisque", "chocolate", "saddlebrown"], labels=['0: No smoker', '1: Light Smoker', '3: Heavy smoker'])
plt.ylabel('')
plt.title('Smoking variable distribution')
plt.subplot(212)
sns.countplot(x=data['tobacco'], hue=data['ten_year_risk_chd'], palette=['forestgreen', 'firebrick']) 
plt.xlabel('Tobacco')
plt.ylabel('Count')
plt.title('Risk CHD by Tobacco consumption')
plt.show()

<b>E) <u>Medical Variables</u></b>

- Medical measures

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
ax = sns.boxplot(data = data[med_measures_vars])
plt.show()

- Map correlation

In [ ]:
correlation = data.corr()
correlation['ten_year_risk_chd'].sort_values(ascending=False)

In [ ]:
fig = plt.figure(figsize=(15,15))
sns.heatmap(correlation, annot=True)

### Part 3. MODEL PREPARATION

<b>A) <u>Feature scaling using 'StandardScaler'</u></b>

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
X = data.drop(['ten_year_risk_chd'], axis=1)
y = data['ten_year_risk_chd']

In [ ]:
scaler = StandardScaler() 
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
X_scaled.head()

In [ ]:
X_scaled.describe()

<b>B) <u>Splitting data (using 'train_test_split')</u></b>

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.3, random_state = 42)

In [ ]:
print("len(X_train) =", len(X_train))
print("len(X_test) =", len(X_test))

<b>C) <u>Resampling imbalanced dataset (using SMOTE algorithm)</u></b>

SMOTE (synthetic minority oversampling technique) is one of the most commonly used oversampling methods to solve the imbalance problem.
It aims to balance class distribution by randomly increasing minority class examples by replicating them.

In [ ]:
print("Before OverSampling, counts of risk_CDH = 0 :", sum(y_train==0))
print("Before OverSampling, counts of risk_CDH = 1 :", sum(y_train==1))

In [ ]:
from imblearn.over_sampling import SMOTE 

sm = SMOTE()
X_train, y_train = sm.fit_sample(X_train, y_train)

In [ ]:
print("After OverSampling, counts of risk_CDH = 0 :", sum(y_train==0))
print("After OverSampling, counts of risk_CDH = 1 :", sum(y_train==1))

<p>SMOTE Algorithm has oversampled the minority instances (risk_CHD = 1) and made it equal to majority class (risk_CHD = 0).</p>
<p>Now, both categories have equal amount of records (2686).</p>

In [ ]:
from lazypredict.Supervised import LazyClassifier

In [ ]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

### Part 4. MACHINE LEARNING MODELS

The algorithms that we will be used are :

- Logistic Regression
- K-Nearest Neighbors
- Decision Trees
- Random Forest Classification
- Support Vector Machine
- Naive Bayes


<b>A) <u>Logistic Regression</u></b>

- Model

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logr = LogisticRegression()
logr.fit(X_train, y_train)
y_pred = logr.predict(X_test)

- Performance

Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [ ]:
cm_logr = confusion_matrix(y_test, y_pred)
cm_logr

Scores

In [ ]:
acc_logr = accuracy_score(y_test, y_pred)
recall_logr = recall_score(y_test, y_pred)
precision_logr = precision_score(y_test, y_pred)
f1_logr = f1_score(y_test, y_pred)

print("Accuracy Score: ", acc_logr)
print("Recall Score: ", recall_logr)
print("Precision Score: ", precision_logr)
print("F1 Score: ", f1_logr)

In [ ]:
print(classification_report(y_test, y_pred, zero_division=1))

ROC curve and AUC

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
y_pred_prob = logr.predict_proba(X_test)[:,1]
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred_prob)

In [ ]:
plt.plot([0, 1], [0, 1], '--')
plt.plot(false_positive_rate, true_positive_rate, label='ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Logistic Regression ROC Curve')
plt.legend()
plt.show()

In [ ]:
roc_auc_score = roc_auc_score(y_test, y_pred_prob)
roc_auc_score

<b>B) <u>K-Nearest Neighbors</u></b>

- Model

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# function to find optimal 'k' (number of neighbors)

def KNeighbors(X_train, X_test, y_train, y_test):
    
    accuracy_scores = []
    
    for i in range(1,20):
        KN = KNeighborsClassifier(n_neighbors=i)
        KN.fit(X_train, y_train)
        y_pred2 = KN.predict(X_test)
        accuracy_scores.append(accuracy_score(y_pred2, y_test))
    
    max_score = max(accuracy_scores)
    optimal_k = accuracy_scores.index(max_score) + 1
    
    print(f"The best accuracy score for KNN model is: {max_score} when n_neighbors = {optimal_k}")

In [ ]:
KNeighbors(X_train, X_test, y_train, y_test)

In [ ]:
classifier = KNeighborsClassifier(n_neighbors = 2)
classifier.fit(X_train, y_train)
y_pred2 = classifier.predict(X_test)

- Performance

Confusion matrix

In [ ]:
cm_knn = confusion_matrix(y_pred2, y_test)
cm_knn

Scores

In [ ]:
accuracy_knn = accuracy_score(y_test, y_pred2)
print("Accuracy Score: ", accuracy_knn)

In [ ]:
print(classification_report(y_test, y_pred2, zero_division=1))

<b>C) <u>Decision tree</u></b>

- Model

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

In [ ]:
# function to fix the maximum number of lead nodes (pruning)

def pruning(X_train, X_test, y_train, y_test):
    
    accuracy_scores = []
    
    for i in range(2,20):
        dt = DecisionTreeClassifier(max_leaf_nodes=i)
        dt.fit(X_train, y_train)
        y_pred3 = dt.predict(X_test)
        accuracy_scores.append(accuracy_score(y_pred3, y_test))
    
    max_score = max(accuracy_scores)
    nodes = accuracy_scores.index(max_score) + 2
    print(f"The best accuracy score for decision tree classifier is: {max_score} when max_leaf_nodes = {nodes}")

In [ ]:
pruning(X_train, X_test, y_train, y_test)

In [ ]:
decision_tree = DecisionTreeClassifier(max_leaf_nodes = 17)
decision_tree.fit(X_train, y_train)
y_pred3 = decision_tree.predict(X_test) 

In [ ]:
plt.figure(figsize = (15,10))
plot_tree(decision_tree, filled = True, feature_names = X.columns, proportion = True)

- Performance

Confusion matrix

In [ ]:
cm_tree = confusion_matrix(y_pred3, y_test)
cm_tree

Scores

In [ ]:
acc_decision_tree = accuracy_score(y_test, y_pred3)
acc_decision_tree

In [ ]:
print(classification_report(y_test, y_pred3, zero_division=1))

<b>D) <u>Random Forest</u></b>

In [ ]:
def forest(X_train, X_test, y_train, y_test):
    
    accuracy_scores = []
    
    for i in range(1,1000):
        rf = RandomForestClassifier(n_estimators=i)
        rf.fit(X_train, y_train)
        y_pred3 = rf.predict(X_test)
        accuracy_scores.append(accuracy_score(y_pred3, y_test))
    
    max_score = max(accuracy_scores)
    estimators = accuracy_scores.index(max_score) + 2
    print(f"The best accuracy score for decision tree classifier is: {max_score} when max_leaf_nodes = {estimators}")

In [ ]:
forest(X_train, X_test, y_train, y_test)

- Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

- Performance

Confusion matrix

In [ ]:
cm_forest = confusion_matrix(y_pred4, y_test)
cm_forest

Scores

In [ ]:
forest_accuracy = accuracy_score(y_pred4, y_test)
forest_accuracy

In [ ]:
print(classification_report(y_test, y_pred4, zero_division=1))

<b>E) <u>Support Vector Machine</u></b>

In [ ]:
from sklearn.svm import SVC

In [ ]:
svc_model = SVC(kernel= 'rbf', max_iter=10000, C=1.0, gamma='auto')
svc_model.fit(X_train, y_train)
y_pred5 = svc_model.predict(X_test)

In [ ]:
cm_svc = confusion_matrix(y_pred5, y_test)
cm_svc

In [ ]:
svc_accuracy = accuracy_score(y_pred5, y_test)
svc_accuracy

In [ ]:
print(classification_report(y_test, y_pred5, zero_division=1))

<b>F) <u>Naive Bayes</u></b>

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
naive_bayes = GaussianNB()
naive_bayes.fit(X_train, y_train)
y_pred6 = naive_bayes.predict(X_test)

In [ ]:
cm_nb = confusion_matrix(y_pred6, y_test)
cm_nb

In [ ]:
nb_accuracy = accuracy_score(y_pred6, y_test)
nb_accuracy

In [ ]:
print(classification_report(y_test, y_pred6, zero_division=1))

### Final results

In [ ]:
data = {'Model':['Logistic Regression','KNN','Decision Tree','SVM','Random Forest','Naive Bayes'],
        'F1 Score':[6.60,12.5,22.7,1.70,13.0,26.0],'Accuracies':[84.89,84.1,74.1,84.7,83.89,81.8],'Recall':[3.40,7.30,24.6,0.89,7.80,20.70],'Precision':[72.70,41.50,21.00,100.00,40.00,35.00]}

# Create DataFrame
df = pd.DataFrame(data)
 
# Print the output.
print(df)

In [ ]:
plt.figure(figsize=(9,6))
sns.barplot(x='Model', y='Accuracies', data = df)
plt.title('Comparison of accuracy of models')
plt.xlabel('model algorithms', fontsize=14)
plt.ylabel('Accuracy', fontsize=14)
plt.show()

In [ ]:

names = ['Logistic Regression','KNN','Decision Tree','Random Forest','SVC', 'Naives Bayes', 'Extreme Gradient Boost']

models = [LogisticRegression(), 
         KNeighborsClassifier(n_neighbors = 14),
         DecisionTreeClassifier(max_leaf_nodes = 2),
         RandomForestClassifier(n_estimators= 5000, random_state= 42),
         SVC(kernel= 'rbf', max_iter=10000, C=1.0, gamma='auto'),
         GaussianNB(), 
         ]

for n, mod in zip(names, models):
    
    mod.fit(X_train, y_train) 
    print(f"Accuracy {n}: {round((mod.score(X_test, y_test)) *100,2)}")

Cross validation of Logistic Regression

In [ ]:
score=cross_val_score(LogisticRegression(),imputed_data.drop('TenYearCHD',axis=1),imputed_data['TenYearCHD'],cv=10)
print(f"After k-fold cross validation score is {score.mean()}")

### STACKING

In [ ]:
from mlxtend.classifier import StackingCVClassifier

In [ ]:
stacking = StackingCVClassifier(classifiers=[xgb,knn,svc],meta_classifier= svc,random_state=42)
scv.fit(X_train,y_train)
scv_predicted = scv.predict(X_test)
scv_conf_matrix = confusion_matrix(y_test, scv_predicted)
scv_acc_score = accuracy_score(y_test, scv_predicted)
print("confusion matrix")
print(scv_conf_matrix)
print("\n")
print("Accuracy of StackingCVClassifier:",scv_acc_score*100,'\n')
print(classification_report(y_test,scv_predicted))